In [ ]:
import hashlib
import os
from bing_image_downloader import downloader

In [ ]:
pan_dulce_types = {'conchita': ['conchita', 'concha'],
                    'quequito': ['quequito',
                    'mantecada',
                    'mantecada de chocolate',
                    'quequito de chocolate'],
                    'puerquito': ['puerquito', 'cochinito', 'marranito', 'puerco'],
                    'elote': ['elote', 'elotito'],
                    'barquillo': ['barquillo', 'cono de crema'],
                    'orejas': ['orejas', 'orejitas'],
                    'pan de muerto': ['pan de muerto'],
                    'rosca de reyes': ['rosca de reyes'],
                    'donas': ['donas'],
                    'trenzas': ['trenzas', 'trenzitas'],
                    'cuernitos': ['cuernitos', 'cuerno'],
                    'chilindrina': ['chilindrina', 'chilindrinas'],
                    'besos': ['besos', 'besitos', 'ojo de buey', 'yoyos'],
                    'niño envuelto': ['niño envuelto']}

In [ ]:
for pan, names in pan_dulce_types.items():
    directory = 'pan_dulce/' + pan
    if not os.path.isdir(directory):
        os.makedirs(directory)
    
    queries = [name + ' pan dulce' for name in names]
    max_limit = int(120 / len(queries) + 25)
    for query in queries:
        downloader.download(query, 
                            limit = max_limit,  
                            output_dir = directory, 
                            adult_filter_off = True, 
                            force_replace = False, 
                            timeout = 10, 
                            verbose = True)
    
    hashes = set()
    for folder in os.listdir(directory):
        folder_name = os.path.join(directory, folder)
        for filename in os.listdir(folder_name):
            path = os.path.join(directory, folder, filename)
            digest = hashlib.sha1(open(path,'rb').read()).digest()
            if digest not in hashes:
                hashes.add(digest)
                os.rename(path, directory + '/' + filename)
            else:
                os.remove(path)

In [ ]:
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

subscription_key = '3320313439744cd1abc098056cb219f7'
search_url = "https://api.bing.microsoft.com/v7.0/images/search"
search_term = "puppies"

In [ ]:
headers = {"Ocp-Apim-Subscription-Key" : subscription_key}
params  = {"q": search_term, "license": "public", "imageType": "photo"}

In [ ]:
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()
thumbnail_urls = [img["thumbnailUrl"] for img in search_results["value"][:16]]

In [ ]:
f, axes = plt.subplots(4, 4)
for i in range(4):
    for j in range(4):
        image_data = requests.get(thumbnail_urls[i+4*j])
        image_data.raise_for_status()
        image = Image.open(BytesIO(image_data.content))        
        axes[i][j].imshow(image)
        axes[i][j].axis("off")
plt.show()